<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Wrangling and Analyze Data

In [1]:
# Import dependencies
import requests
import os
import json
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
path_csv = 'https://raw.githubusercontent.com/lustraka/Data_Analysis_Workouts/main/Analyse_Twitter_Data/'
dfa = pd.read_csv(path_csv+'twitter-archive-enhanced.csv')
dfa.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
url_tsv = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url_tsv)
with open('image-predictions.tsv', 'wb') as file:
  file.write(r.content)
dfi = pd.read_csv('image-predictions.tsv', sep='\t')
dfi.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
from timeit import default_timer as timer

count = 0
fails_dict =  {}
start = timer()

if 'tweet_json.txt' in os.listdir():
  os.remove('tweet_json.txt')

with open('tweet_json.txt', 'a') as file:
  for tweet_id in dfa.tweet_id.values:
    count += 1
    print(str(count) + ': ' + str(tweet_id))
    try:
      status = api.get_status(tweet_id, tweet_mode='extended')._json
      print("Success")
      file.write(json.dumps(status, ensure_ascii=False)+'\n')
    except tweepy.TweepError as e:
      print('Fail')
      fails_dict[tweet_id] = e
      pass
    except e:
      print('Fail', e)
end = timer()
print(f'Elapsed time: {end - start}')
print(fails_dict)

Data gathered form Twitter API:

| Attribute | Type | Description |
| --- | :-: | --- |
| id | int | The integer representation of unique identifier for this Tweet |
| retweet_count | int | Number of times this Tweet has been retweeted. |
| favorite_count | int | *Nullable*. Indicates approximately how many times this tweet has been liked by Twitter users. |

Reference: [Tweepy docs: Tweet Object](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)



In [ ]:
df_tweets = []
with open('tweet_json.txt', 'r') as file:
  line = file.readline()
  while line:
    status = json.loads(line)
    df_tweets.append({'tweet_id': status['id'], 'retweet_count': status['retweet_count'], 'favorite_count': status['favorite_count']})
    line = file.readline()
dft = pd.DataFrame(df_tweets)
dft.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7164,34458
1,892177421306343426,5385,29870
2,891815181378084864,3551,22490
3,891689557279858688,7383,37671
4,891327558926688256,7922,35988


In [ ]:
# Store dataframes for further processing if necessary


In [2]:
# Download the database
url_db = 'https://github.com/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/weratedogsdata.db?raw=true'
r = requests.get(url_db)
with open('weratedogsdata.db', 'wb') as file:
  file.write(r.content)

from sqlalchemy import create_engine
# Create SQLAlchemy Engine and connect to the database
engine = create_engine('sqlite:///weratedogsdata.db')

# Read dataframes from SQlite database
dfa = pd.read_sql('SELECT * FROM dba', engine)
dfi = pd.read_sql('SELECT * FROM dbi', engine)
dft = pd.read_sql('SELECT * FROM dbt', engine)

In [3]:
dfa.shape, dfi.shape, dft.shape

((2356, 17), (2075, 12), (2327, 3))

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### The archive `twitter_archive_enhanced.csv` (alias `dba`)
> "I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too. You'll need to assess and clean these columns if you want to use them for analysis and visualization."

In [4]:
dfa.sample(15)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2174,669216679721873412,NaN,NaN,2015-11-24 18:11:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Clarq. He's a golden Quetzalcoatl. Cla...,NaN,NaN,None,https://twitter.com/dog_rates/status/669216679...,8,10,Clarq,None,None,None,None
326,833826103416520705,NaN,NaN,2017-02-20 23:50:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Charlie. She asked u to change the channe...,NaN,NaN,None,https://twitter.com/dog_rates/status/833826103...,13,10,Charlie,None,None,None,None
2272,667491009379606528,NaN,NaN,2015-11-19 23:53:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Two dogs in this one. Both are rare Jujitsu Py...,NaN,NaN,None,https://twitter.com/dog_rates/status/667491009...,7,10,None,None,None,None,None
201,853760880890318849,NaN,NaN,2017-04-17 00:03:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Alice. I'm told she enjoys car ri...,NaN,NaN,None,https://twitter.com/dog_rates/status/853760880...,12,10,Alice,None,None,None,None
410,823322678127919110,NaN,NaN,2017-01-23 00:13:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wyatt. He's got the fastest paws in th...,NaN,NaN,None,https://twitter.com/dog_rates/status/823322678...,11,10,Wyatt,None,None,None,None
1709,680609293079592961,NaN,NaN,2015-12-26 04:41:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Brody. That is his chair. He loves his...,NaN,NaN,None,https://twitter.com/dog_rates/status/680609293...,9,10,Brody,None,None,None,None
75,878281511006478336,NaN,NaN,2017-06-23 16:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Shadow. In an attempt to reach maximum zo...,NaN,NaN,None,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
1815,676613908052996102,NaN,NaN,2015-12-15 04:05:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is the saddest/sweetest/best picture I've...,NaN,NaN,None,https://twitter.com/dog_rates/status/676613908...,12,10,the,None,None,None,None
247,845459076796616705,NaN,NaN,2017-03-25 02:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Here's a heartwarming scene of ...,7.562885e+17,4.196984e+09,2016-07-22 00:43:32 +0000,https://twitter.com/dog_rates/status/756288534...,12,10,None,None,None,None,None
473,816336735214911488,NaN,NaN,2017-01-03 17:33:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dudley. He found a flower and now he's...,NaN,NaN,None,https://twitter.com/dog_rates/status/816336735...,11,10,Dudley,None,None,None,None


In [5]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [19]:
for col in dfa.columns[[10,11,13,14,15,16]]:
  print(dfa[col].unique())

[  13   12   14    5   17   11   10  420  666    6   15  182  960    0
   75    7   84    9   24    8    1   27    3    4  165 1776  204   50
   99   80   45   60   44  143  121   20   26    2  144   88]
[ 10   0  15  70   7  11 150 170  20  50  90  80  40 130 110  16 120   2]
['None' 'doggo']
['None' 'floofer']
['None' 'pupper']
['None' 'puppo']


Observations

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | tweet_id | 2356 | 2356 | int64  | |
| 1 | in_reply_to_status_id | 78 | 77 | float64 | majority values missing |
| 2 | in_reply_to_user_id | 78 | 31 | float64 | majority values missing |
| 3 | timestamp | 2356 | 2356 | object | object $\to$ datetime | 
| 4 | source | 2356 | 4 | object | |
| 5 | text | 2356 | 2356 | object | |
| 6 | retweeted_status_id | 181 | 181 | float64 | indicates not original tweet |
| 7 | retweeted_status_user_id | 181 | 25 | float64 | indicates not original tweet |
| 8 | retweeted_status_timestamp | 181 | 181 | object | indicates not original tweet |
| 9 | expanded_urls | 2297 | 2218 | object | missing values |
| 10 | rating_numerator | 2356 | 40 | int64  | entries with numerator $> 20$ may be incorrect  |
| 11 | rating_denominator | 2356 | 18 | int64  | entries with denominator $\neq 10$ may be incorrect |
| 12 | name | 2356 | 957 | object | incorrect ('the') or missing values (also None instead of NaN) |
| 13 | doggo | 2356 | 2 | object | a value as a column |
| 14 | floofer | 2356 | 2 | object | a value as a column |
| 15 | pupper | 2356 | 2 | object | a value as a column |
| 16 | puppo | 2356 | 2 | object | a value as a column |

Source: visual and programmatic assessment

```python
# #, Variable, Non-Null (Count), Dtype:
dfa.info()
# Nunique:
dfa.nunique()
# Check unique values
for col in dfa.columns[[10,11,13,14,15,16]]:
  print(dfa[col].unique())
```

#### Check ratings where `rating_numerator` $ > 20$ or  `rating_denomiator` $\neq 10$
Code used:
```python
# Show the whole text
pd.options.display.max_colwidth = None

# Show tweets with possibly incorrect rating I
dfa.loc[dfa.rating_numerator > 20, ['text', 'rating_numerator', 'rating_denominator']]

# Show tweets with possibly incorrect rating II
dfa.loc[dfa.rating_denominator != 10, ['text', 'rating_numerator', 'rating_denominator']]

```
In cases where users used float numbers, such as 9.75/10 or 11.27/10, we will use the floor rounding, i.e. 9/10 or 11/10 respectively. We will correct only those rating which were incorrectly identified in the text. Ratings with weird values used in the text are left unchanged cos they're good dogs Brent.

Results:

In [32]:
# Show the whole text
pd.options.display.max_colwidth = None

# Fill dict with key = index and value = correct rating
incorrect_rating = {313 : '13/10', 340 : '9/10', 763: '11/10', 1717 : '11/10', 313 : '13/10', 784 : '14/10', 1165 : '13/10', 1202 : '11/10', 1662 : '10/10', 2335 : '9/10'}

# Indicate tweets with missing rating
missing_rating = [342, 516]

# Show tweet with incorrectly identified rating
dfa.loc[list(incorrect_rating.keys()), ['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
313,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
340,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
1717,Here's a sleepy Christmas pupper 11/10 https://t.co/KXg0f8GNQ9,11,10
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1662,This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,7,11
2335,This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,1,2


### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization